### Parts adapted from optuna code

In [151]:
MAX_LEN = 26
LR = 0.0001
BATCH_SIZE = 256

In [152]:
import os
import numpy as np
import pandas as pd
import deepdish as dd

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import CometLogger

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader

import optuna
from optuna.integration import PyTorchLightningPruningCallback
from torchnlp.encoders.text import CharacterEncoder, StaticTokenizerEncoder

In [153]:
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, roc_auc_score, roc_curve, matthews_corrcoef, plot_confusion_matrix
from collections import defaultdict
import seaborn as sns

In [155]:
def split(X, y, split):
    X_train, y_train, X_test, y_test = [], [], [], []
    hist = [[], [], []]
    for i, val in enumerate(y):
        if val < 0.1:
            hist[0].append((X[i], val))
        elif val <= 0.90:
            hist[1].append((X[i], val))
        else:
            hist[2].append((X[i], val))
            
    for h in hist:
        np.random.shuffle(h)
        limit = int(len(h) * split)
        d1, d2 = h[:limit], h[limit:]
        for pair in d1:
            X_train.append(pair[0])
            y_train.append(pair[1])
        for pair in d2:
            X_test.append(pair[0])
            y_test.append(pair[1])
            
    return X_train, y_train, X_test, y_test

In [175]:
class RNNRegression(pl.LightningModule):
    def __init__(self, trial, vocab_size):
        super(RNNRegression, self).__init__()
        self.out_predictions = []
        self.trial = trial
        
        self.emb_dim = trial.suggest_categorical('emb_dim', [16, 32, 64])
        self.rnn_num_layers = trial.suggest_int('rnn_num_layers', 1, 4)
        self.rnn_dropout = trial.suggest_float('rnn_dropout', 0.1, 0.5)
        self.rnn_hidden_size = trial.suggest_categorical('rnn_hidden_size', (32, 64, 100))
        self.classifier_dropout = trial.suggest_float('classifier_dropout', 0.1, 0.5)        
        self.batch_size = trial.suggest_categorical('batch_size', (64, 128, 256))
        self.lr = self.trial.suggest_loguniform('lr', 1e-4, 1e-1)
        
        print('Training RNN with: %d LSTM layers -- %d LSTM hidden size -- %.2f LSTM dropout -- %d embedding size -- %.2f classifier dropout -- %d batch size -- %.3f lr' % (self.rnn_num_layers, self.rnn_hidden_size, self.rnn_dropout, self.emb_dim, self.classifier_dropout, self.batch_size, self.lr))
        
        self.embeddings = nn.Embedding(vocab_size, self.emb_dim)
        self.rnn = nn.LSTM(input_size=self.emb_dim, hidden_size=self.rnn_hidden_size, num_layers=self.rnn_num_layers, dropout=self.rnn_dropout, bidirectional=True, batch_first=True)
        self.dropout = nn.Dropout(self.classifier_dropout)
        self.linear = nn.Linear(2 * self.rnn_hidden_size, 1)
        

    def forward(self, x):
        x = self.embeddings(x.long())
        x, _ = self.rnn(x)
        cls_token_emb = x[:, 0, :]
        x = self.dropout(cls_token_emb)
        x = self.linear(x)
        return x
    

    def training_step(self, batch, batch_idx):
        x, y = batch
        train_out = self(x)
        loss = F.mse_loss(torch.squeeze(train_out), y)
        self.log('train_loss', loss)
        return loss

    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), self.lr)
    
            
    def validation_step(self, batch, batch_idx):
        x, y = batch
        val_out = self(x)
        val_loss = F.mse_loss(torch.squeeze(val_out), y)
        self.log('val_loss', val_loss)
        return val_loss
    
            
    def test_step(self, batch, batch_idx):
        x, y = batch
        test_out = self(x)
        self.out_predictions.append(test_out)
        test_loss = F.mse_loss(torch.squeeze(test_out), y)
        self.log('test_loss', test_loss)
        return test_loss
    
    
    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            list(zip(X_train, y_train)),
            batch_size=self.batch_size,
            shuffle=True,
        )
    

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            list(zip(X_val, y_val)),
            batch_size=self.batch_size,
            shuffle=False,
        )
    
    
    def test_dataloader(self):
        return torch.utils.data.DataLoader(
            list(zip(X_test, y_test)),
            batch_size=self.batch_size,
            shuffle=False,
        )

In [158]:
train = dd.io.load('../splits/train.h5')
val = dd.io.load('../splits/val.h5')
test = dd.io.load('../splits/test.h5')
y_train = np.load('../splits/y_train.npy')
y_val = np.load('../splits/y_val.npy')
y_test = np.load('../splits/y_test.npy')

In [159]:
y_train = y_train * 100
y_val = y_val * 100
y_test = y_test * 100

In [160]:
all_seqs = set()

In [161]:
all_seqs.update([item[0] for item in train])
all_seqs.update([item[1] for item in train])
all_seqs.update([item[0] for item in val])
all_seqs.update([item[1] for item in val])
all_seqs.update([item[0] for item in test])
all_seqs.update([item[1] for item in test])

In [162]:
encoder = CharacterEncoder(all_seqs)

In [163]:
encoder.vocab

['<pad>', '<unk>', '</s>', '<s>', '<copy>', 'A', 'C', 'T', 'G']

In [164]:
enc_dict = dict(zip(encoder.vocab, range(len(encoder.vocab))))

In [165]:
from torchnlp.encoders.text.default_reserved_tokens import DEFAULT_SOS_INDEX
from torchnlp.encoders.text.default_reserved_tokens import DEFAULT_EOS_INDEX
from torchnlp.encoders.text.default_reserved_tokens import DEFAULT_PADDING_INDEX

In [166]:
def encode_for_rnn(seq, max_len):
    # Default padding index is zero for the character encoder
    nucl_dict = {'A': enc_dict['A'], 'C': enc_dict['C'], 'G': enc_dict['G'], 'T': enc_dict['T']}
    mat = np.zeros(max_len, dtype=int)
    
    for i, nucl in enumerate(seq):
        mat[i] = nucl_dict[nucl]
    return mat

def encode_pair_for_rnn(seq1, seq2, max_len):
    enc1 = encode_for_rnn(seq1, max_len)
    enc2 = encode_for_rnn(seq2, max_len)
    return np.hstack((np.array([DEFAULT_SOS_INDEX]), enc1, np.array([DEFAULT_EOS_INDEX]), enc2, np.array([DEFAULT_EOS_INDEX])))

In [167]:
X_train = [encode_pair_for_rnn(item[0], item[1], MAX_LEN) for item in train]
X_val = [encode_pair_for_rnn(item[0], item[1], MAX_LEN) for item in val]
X_test = [encode_pair_for_rnn(item[0], item[1], MAX_LEN) for item in test]

In [168]:
X_train = np.array(X_train, dtype=np.dtype('d'))
X_val = np.array(X_val, dtype=np.dtype('d'))
X_test = np.array(X_test, dtype=np.dtype('d'))

In [169]:
torch.set_default_tensor_type(torch.DoubleTensor)
torch.set_default_dtype(torch.double)

In [170]:
class MetricsCallback(pl.Callback):
    """PyTorch Lightning metric callback."""

    def __init__(self):
        super().__init__()
        self.metrics = []

    def on_validation_end(self, trainer, pl_module):
        self.metrics.append(trainer.callback_metrics)

In [172]:
DIR = os.getcwd()
MODEL_DIR = os.path.join(DIR, "model_saved")

In [173]:
def objective(trial):
    # Filenames for each trial must be made unique in order to access each checkpoint.
    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        os.path.join(MODEL_DIR, "trial_{}".format(trial.number), "{epoch}"), monitor="val_loss"
    )
    

    # The default logger in PyTorch Lightning writes to event files to be consumed by
    # TensorBoard. We don't use any logger here as it requires us to implement several abstract
    # methods. Instead we setup a simple callback, that saves metrics from each validation step.
    metrics_callback = MetricsCallback()
    trainer = pl.Trainer(
        max_epochs=5,
        gpus=[0],
        callbacks=[checkpoint_callback, metrics_callback, PyTorchLightningPruningCallback(trial, monitor="val_loss")],
    )

    model = RNNRegression(trial, len(encoder.vocab))
    trainer.fit(model)

    return metrics_callback.metrics[-1]["val_loss"].item()

In [ ]:
pruner = optuna.pruners.HyperbandPruner()

study = optuna.create_study(direction="minimize", pruner=pruner)
study.optimize(objective, n_trials=50)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2021-01-13 16:58:59,552] A new study created in memory with name: no-name-6ebcb3d1-eecc-46f8-9381-54c3a3a7c97d
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
C:\ProgramData\Anaconda3\envs\pt-light-1.7.0\lib\site-packages\torch\nn\modules\rnn.py:58: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2543327381422479 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "

  | Name       | Type      | Params
-----------------------------------------
0 | embeddings | Embedding | 576   
1 | rnn        | LSTM      | 66.6 K
2 | dropout    | Dropout   | 0     
3 | linear     | Linear    | 129   
-----------------------------------------
67.3 K    Trainable params
0         Non-trainable params
67.3 K    Total params


Training RNN with: 1 LSTM layers -- 64 LSTM hidden size -- 0.25 LSTM dropout -- 64 embedding size -- 0.20 classifier dropout -- 256 batch size -- 0.022 lr


Validation sanity check: |                                                                       | 0/? [00:00<…

Training: |                                                                                      | 0/? [00:00<…

Validating: |                                                                                    | 0/? [00:00<…

...

Validating: |                                                                                    | 0/? [00:00<…

[I 2021-01-19 02:55:37,982] Trial 41 pruned. Trial was pruned at epoch 3.
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name       | Type      | Params
-----------------------------------------
0 | embeddings | Embedding | 288   
1 | rnn        | LSTM      | 248 K 
2 | dropout    | Dropout   | 0     
3 | linear     | Linear    | 129   
-----------------------------------------
249 K     Trainable params
0         Non-trainable params
249 K     Total params


Training RNN with: 3 LSTM layers -- 64 LSTM hidden size -- 0.13 LSTM dropout -- 32 embedding size -- 0.21 classifier dropout -- 128 batch size -- 0.003 lr


Validation sanity check: |                                                                       | 0/? [00:00<…

Training: |                                                                                      | 0/? [00:00<…